In [1]:
import pandas as pd

df=pd.read_csv('train.csv')

#df.plot.hist()
labels= df.Label
#print(labels)

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import plot_model, to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score

import glob
import imageio
import numpy as np
import os.path as path
from datetime import datetime

# Image processing
IMAGE_PATH='train_images'

image_input_files = sorted(glob.glob(path.join(IMAGE_PATH, '*.png')))
images = [imageio.imread(im) for im in image_input_files]
images = np.asarray(images)
#images = images[..., None]
n_images = len(image_input_files)

#print(n_images)

#images = images.reshape(,180,1500,3)
image_size = np.asarray([images.shape[0], images.shape[1], images.shape[2]])
print(image_size)
#images = np.expand_dims(images,axis=0)
#print(image_size)
images = images/256


Using TensorFlow backend.


[2528  128  128]


In [11]:
from sklearn.model_selection import train_test_split

num_classes = 6
img_rows = 128
img_cols = 128

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.30)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
    
y_train=to_categorical(y_train, num_classes)
y_test=to_categorical(y_test, num_classes)

print(x_train)
print(x_test)

print(y_train)
print(y_test)

[[[[0.65625   ]
   [0.68359375]
   [0.68359375]
   ...
   [0.55859375]
   [0.609375  ]
   [0.61328125]]

  [[0.72265625]
   [0.65625   ]
   [0.6640625 ]
   ...
   [0.59765625]
   [0.578125  ]
   [0.5859375 ]]

  [[0.65234375]
   [0.69140625]
   [0.66796875]
   ...
   [0.578125  ]
   [0.56640625]
   [0.54296875]]

  ...

  [[0.63671875]
   [0.65625   ]
   [0.71875   ]
   ...
   [0.5546875 ]
   [0.6015625 ]
   [0.6484375 ]]

  [[0.6640625 ]
   [0.6796875 ]
   [0.74609375]
   ...
   [0.51171875]
   [0.57421875]
   [0.58203125]]

  [[0.66015625]
   [0.66796875]
   [0.7265625 ]
   ...
   [0.62109375]
   [0.6015625 ]
   [0.625     ]]]


 [[[0.765625  ]
   [0.7578125 ]
   [0.73828125]
   ...
   [0.765625  ]
   [0.78125   ]
   [0.7890625 ]]

  [[0.76171875]
   [0.7578125 ]
   [0.73828125]
   ...
   [0.76171875]
   [0.77734375]
   [0.78125   ]]

  [[0.75390625]
   [0.75      ]
   [0.734375  ]
   ...
   [0.74609375]
   [0.76171875]
   [0.765625  ]]

  ...

  [[0.89453125]
   [0.88671875]
   [0.8

In [9]:
from keras.optimizers import RMSprop

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(128,128,1)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

model.summary()

batch_size=100
epochs=50

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128, 128, 512)     1024      
_________________________________________________________________
dropout_11 (Dropout)         (None, 128, 128, 512)     0         
_________________________________________________________________
dense_16 (Dense)             (None, 128, 128, 512)     262656    
_________________________________________________________________
dropout_12 (Dropout)         (None, 128, 128, 512)     0         
_________________________________________________________________
dense_17 (Dense)             (None, 128, 128, 6)       3078      
Total params: 266,758
Trainable params: 266,758
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_17 to have 4 dimensions, but got array with shape (1769, 6)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()